In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
data_folder_name = '데이터'
data_path = os.path.abspath(data_folder_name)

In [3]:
file_accom_raw = os.path.join(data_path, '제주관광공사_제주관광정보시스템(VISIT JEJU)_숙박콘텐츠_20250307.CSV')

In [4]:
file_accom_clean = os.path.join(data_path, 'golden_compass_accommodation_clean.csv')

print(f"원본 파일 경로: {file_accom_raw}")
print(f"저장할 파일 경로: {file_accom_clean}")

원본 파일 경로: c:\공모전\데이터\제주관광공사_제주관광정보시스템(VISIT JEJU)_숙박콘텐츠_20250307.CSV
저장할 파일 경로: c:\공모전\데이터\golden_compass_accommodation_clean.csv


원본 데이터 로드 및 탐색

In [5]:
try:
    df_raw = pd.read_csv(file_accom_raw, encoding='cp949')
    print("--- 숙소 원본 데이터 로드 완료 ---")

    filter_cols = [
        '콘텐츠명', '등급', '장애인전용객실여부', '애완동물동반허용여부', 
        '조식제공여부', '부대시설기타', 'LATE체크인여부', '셔틀버스운행여부'
    ]
    
    available_cols = [col for col in filter_cols if col in df_raw.columns]
    df_clean = df_raw[available_cols].copy()

    print("\n--- [1] 원본 데이터 결측값(NaN) 개수 확인 ---")
    print(df_clean.isnull().sum())
except Exception as e:
    print(f"파일 로드 중 오류 발생: {e}")

--- 숙소 원본 데이터 로드 완료 ---

--- [1] 원본 데이터 결측값(NaN) 개수 확인 ---
콘텐츠명             0
등급            1173
장애인전용객실여부     1234
애완동물동반허용여부    1138
조식제공여부        1180
부대시설기타        1315
LATE체크인여부     1208
셔틀버스운행여부      1225
dtype: int64


결측치/이상치 제거(데이터 클리닝)

In [6]:
if '콘텐츠명' in df_clean.columns:
    print(f"원본 '콘텐츠명' 개수 (중복 포함): {len(df_clean)}")
    
    df_clean = df_clean.dropna(subset=['콘텐츠명'])
    print(f"-> '콘텐츠명' 결측치(NaN) 행 제거 후: {len(df_clean)}")

    is_numeric_mask = df_clean['콘텐츠명'].str.isnumeric().fillna(False)
    df_clean = df_clean[~is_numeric_mask]
    print(f"-> 숫자로만 된 '콘텐츠명' 이상치 제거 후: {len(df_clean)}")

    df_clean = df_clean.drop_duplicates(subset=['콘텐츠명'], keep='first')
    print(f"-> '콘텐츠명' 중복값 제거 후 (고유 숙소 개수): {len(df_clean)}")
else:
    print("Warning: '콘텐츠명' 컬럼이 원본에 없습니다.")

원본 '콘텐츠명' 개수 (중복 포함): 1397
-> '콘텐츠명' 결측치(NaN) 행 제거 후: 1397
-> 숫자로만 된 '콘텐츠명' 이상치 제거 후: 1394
-> '콘텐츠명' 중복값 제거 후 (고유 숙소 개수): 1382


In [7]:

if '등급' in df_clean.columns:
    df_clean['등급'] = pd.to_numeric(df_clean['등급'], errors='coerce')
    df_clean['등급'] = df_clean['등급'].fillna(0)
    df_clean['등급'] = df_clean['등급'].astype(int)

def clean_yes_no_to_ox(column):
    filled_col = column.fillna('n')
    str_col = filled_col.astype(str).str.lower()
    std_col = np.where(str_col.isin(['y', 'true', '1', '1.0']), 'y', 'n')
    return np.where(std_col == 'y', 'O', 'X')


In [8]:

if '장애인전용객실여부' in df_clean.columns:
    df_clean['장애인전용객실여부'] = clean_yes_no_to_ox(df_clean['장애인전용객실여부'])

In [9]:

if '애완동물동반허용여부' in df_clean.columns:
    df_clean['애완동물동반허용여부'] = clean_yes_no_to_ox(df_clean['애완동물동반허용여부'])


In [10]:

if '조식제공여부' in df_clean.columns:
    df_clean['조식제공여부'] = clean_yes_no_to_ox(df_clean['조식제공여부'])


In [11]:

if 'LATE체크인여부' in df_clean.columns:
    df_clean['LATE체크인여부'] = clean_yes_no_to_ox(df_clean['LATE체크인여부'])


In [12]:

if '셔틀버스운행여부' in df_clean.columns:
    df_clean['셔틀버스운행여부'] = clean_yes_no_to_ox(df_clean['셔틀버스운행여부'])


In [13]:

if '부대시설기타' in df_clean.columns:
    df_clean['부대시설기타'] = df_clean['부대시설기타'].fillna('정보 없음')


In [14]:
df_clean.head()

,콘텐츠명,등급,장애인전용객실여부,애완동물동반허용여부,조식제공여부,부대시설기타,LATE체크인여부,셔틀버스운행여부
0,나쿠펜다 제주,0,X,X,X,정보 없음,X,X
1,제주엘루이호텔,0,X,X,X,정보 없음,X,X
2,아인스호텔,0,X,X,X,정보 없음,X,X
3,씨스테이호텔,0,X,X,X,정보 없음,X,X
4,썬라이즈호텔,0,X,X,X,정보 없음,X,X


데이터 클리닝 결과 확인

In [15]:
if '등급' in df_clean.columns:
    print("\n'등급' (int):")
    print(df_clean['등급'].value_counts().sort_index())

if '장애인전용객실여부' in df_clean.columns:
    print("\n'장애인전용객실여부' (O/X):")
    print(df_clean['장애인전용객실여부'].value_counts())

if '애완동물동반허용여부' in df_clean.columns:
    print("\n'애완동물동반허용여부' (O/X):")
    print(df_clean['애완동물동반허용여부'].value_counts())

if '조식제공여부' in df_clean.columns:
    print("\n'조식제공여부' (O/X):")
    print(df_clean['조식제공여부'].value_counts())

if 'LATE체크인여부' in df_clean.columns:
    print("\n'LATE체크인여부' (O/X):")
    print(df_clean['LATE체크인여부'].value_counts())

if '셔틀버스운행여부' in df_clean.columns:
    print("\n'셔틀버스운행여부' (O/X):")
    print(df_clean['셔틀버스운행여부'].value_counts())


'등급' (int):
등급
0    1316
1       6
2      12
3      15
4      19
5      14
Name: count, dtype: int64

'장애인전용객실여부' (O/X):
장애인전용객실여부
X    1343
O      39
Name: count, dtype: int64

'애완동물동반허용여부' (O/X):
애완동물동반허용여부
X    1307
O      75
Name: count, dtype: int64

'조식제공여부' (O/X):
조식제공여부
X    1303
O      79
Name: count, dtype: int64

'LATE체크인여부' (O/X):
LATE체크인여부
X    1237
O     145
Name: count, dtype: int64

'셔틀버스운행여부' (O/X):
셔틀버스운행여부
X    1346
O      36
Name: count, dtype: int64


데이터 파일 저장

In [17]:

file_accom_clean = os.path.join(data_path, 'golden_compass_accommodation_clean.csv')

try:
    df_clean.to_csv(file_accom_clean, index=False, encoding='utf-8-sig')
    print(f"'{file_accom_clean}' 경로에 숙소 필터용 파일 저장 성공.")
except Exception as e:
    print(f"파일 저장 중 오류 발생: {e}")

'c:\공모전\데이터\golden_compass_accommodation_clean.csv' 경로에 숙소 필터용 파일 저장 성공.
